In [79]:
import math
import numpy as np

In [80]:
f = open("input")
asteroid_map = f.read()
f.close()
#asteroid_map='.#..##.###...#######\n##.############..##.\n.#.######.########.#\n.###.#######.####.#.\n#####.##.#.##.###.##\n..#####..#.#########\n####################\n#.####....###.#.#.##\n##.#################\n#####.##.###..####..\n..######..##.#######\n####.##.####...##..#\n.#####..#.######.###\n##...#.##########...\n#.##########.#######\n.####.#.###.###.#.##\n....##.##.###..#####\n.#.#.###########.###\n#.#.#.#####.####.###\n###.##.####.##.#..##\n'
#asteroid_map='.#..#\n.....\n#####\n....#\n...##\n'
#asteroid_map='......#.#.\n#..#.#....\n..#######.\n.#.#.###..\n.#..#.....\n..#....#.#\n#..#....#.\n.##.#..###\n##...#..#.\n.#....####\n'
#asteroid_map='#.#...#.#.\n.###....#.\n.#....#...\n##.#.#.#.#\n....#.#.#.\n.##..###.#\n..#...##..\n..##....##\n......#...\n.####.###.\n'
#asteroid_map='.#..#..###\n####.###.#\n....###.#.\n..###.##.#\n##.##.#.#.\n....###..#\n..#.#..#.#\n#..#.#.###\n.##...##.#\n.....#.#..\n'
asteroid_map = asteroid_map.split("\n")
asteroid_map.pop(-1)
#asteroid_map

''

In [81]:
#In order to label the asteroids so it's easier to work with them, build
#a list of tuples containing the coordinates of an asteroid and it's id.
#The coordinates are stored in a (row, column) format
next_id=0
asteroids = []
for row in range(len(asteroid_map)):
    for column in range(len(asteroid_map[row])):
        if asteroid_map[row][column]=="#":
            asteroids.append(((row, column), next_id))
            next_id+=1

In [82]:
#Build the graph
asteroid_graph=[]
for i in range(len(asteroids)):
    for j in range(i+1, len(asteroids)):
        asteroid_0 = asteroids[i][0]
        asteroid_1 = asteroids[j][0]
        asteroid_0_id = asteroids[i][1]
        asteroid_1_id = asteroids[j][1]
        
        #Get the angle between the line between the asteroids and
        #the horizon
        hypotenuse_length = math.sqrt(pow(asteroid_0[0]-asteroid_1[0], 2) + \
                                      pow(asteroid_0[1]-asteroid_1[1], 2))
        distance=hypotenuse_length
        #Get the projection of the higher asteroid over the horizontal line. If
        #they are at the same height, the angle is 0 or 180, but for practical
        #purposes I will always use 0 as the result is the same. In order to measure 
        #the conterclockwise angle, calculate 180-angle degrees if the higher asteroid
        #is to the left of the other 
        if asteroid_0[0]==asteroid_1[0]:
            if asteroid_0[1]<asteroid_1[1]:
                asteroid_graph.append(((asteroid_0, asteroid_0_id), (asteroid_1, asteroid_1_id), 0, distance))
                asteroid_graph.append(((asteroid_1, asteroid_1_id), (asteroid_0, asteroid_0_id), 180, distance))
            else:
                asteroid_graph.append(((asteroid_0, asteroid_0_id), (asteroid_1, asteroid_1_id), 180, distance))
                asteroid_graph.append(((asteroid_1, asteroid_1_id), (asteroid_0, asteroid_0_id), 0, distance))
            continue
        higher_asteroid = asteroid_0 if asteroid_0[0]<asteroid_1[0] else asteroid_1
        lower_asteroid = asteroid_0 if asteroid_0[0]>asteroid_1[0] else asteroid_1
        #rightmost_asteroid = asteroid_0 if asteroid_0[1]>=asteroid_1[1] else asteroid_1
        leftmost_asteroid = asteroid_0 if asteroid_0[1]<=asteroid_1[1] else asteroid_1
        higher_asteroid_id = asteroid_0_id if higher_asteroid==asteroid_0 else asteroid_1_id
        lower_asteroid_id = asteroid_0_id if lower_asteroid==asteroid_0 else asteroid_1_id

        horizontal_projection = (lower_asteroid[0],higher_asteroid[1])
            
        far_side_length = horizontal_projection[0]-higher_asteroid[0]
        near_side_length = abs(horizontal_projection[1]-lower_asteroid[1])
        
        angle_0_to_1=angle=math.degrees(math.atan2(asteroid_0[0]-asteroid_1[0],\
                                                   asteroid_0[1]-asteroid_1[1]))
        angle_1_to_0=angle=math.degrees(math.atan2(asteroid_1[0]-asteroid_0[0],\
                                                   asteroid_1[1]-asteroid_0[1]))
        
        asteroid_graph.append(((asteroid_0, asteroid_0_id), (asteroid_1, asteroid_1_id), angle_0_to_1, distance))
        asteroid_graph.append(((asteroid_1, asteroid_1_id), (asteroid_0, asteroid_0_id), angle_1_to_0, distance))

In [83]:
connections = {}
for asteroid in asteroids:
    connections[asteroid[1]]=[i for i in asteroid_graph if i[0][0]==asteroid[0]]

In [84]:
visible=[]
for asteroid in asteroids:
    #print(asteroid)
    visible_asteroids=0
    #Iterate over the graph nodes starting by the ones closer to this node. To 
    #get those, first filter the connections from this node to the others and use
    #only those
    reduced_graph = [i for i in connections[asteroid[1]]]
    reduced_graph.sort(key=lambda a: a[3])
    #Now, for each node in this graph, count 1 visible asteroid, then, search the complete
    #graph for the nodes that have connections with the node we just counted, and remove
    #those that have the same angle with the counted node as the counted node has with the
    #current node
    while True:
        try:
            connection=reduced_graph.pop(0)
            visible_asteroids+=1
            #Search the connections of the counted asteroid
            to_remove=[]
            for c in connections[connection[1][1]]:
                if c[2] == connection[2]:
                    to_remove.append(c[1])
            reduced_graph=[i for i in reduced_graph if i[1] not in to_remove]
        except Exception as e:
            visible.append(visible_asteroids)
            break
#Get the best asteroid
print(asteroids[np.argmax(visible)])
np.max(visible)

((22, 17), 258)


288

In [85]:
#We know 288 asteroids are visible from the station so the 200th asteroid will be
#destroyed in the first turn. However, if this was not the case, a simple method to
#apply this same solution would be to apply it in the turn where the 200th asteroid 
#will be destroyed, and ignoring the other turns
#Get the visible asteroids from ours, which has id 258
visible_asteroids=[]
reduced_graph = [i for i in connections[258]]
reduced_graph.sort(key=lambda a: a[3])
while True:
    try:
        connection=reduced_graph.pop(0)
        visible_asteroids.append(connection)
        #Search the connections of the counted asteroid
        to_remove=[]
        for c in connections[connection[1][1]]:
            if c[2] == connection[2]:
                to_remove.append(c[1])
        reduced_graph=[i for i in reduced_graph if i[1] not in to_remove]
    except Exception as e:
        break
#connections[258].sort(key=lambda a: a[2])
#connections[258]
#The ideal solution would be to just sort the asteroids based on the angle in relation 
#to the horizon of a ray from our asteroid. However, as the ray starts at 90, moves to
#0, then makes its way from 0 to -180, and then from 180 to 90, I have to sort 3 lists
#and then join them into one again
#Also, the Y is inverted so the signs of all of that are inverted
ninety_to_180=[a for a in visible_asteroids if 90<=a[2]<=180]
minus_one_hundred_and_eighty_to_0=[a for a in visible_asteroids if -180<=a[2]<=0]
zero_to_90=[a for a in visible_asteroids if 0<a[2]<90]
ninety_to_180.sort(key=lambda a:a[2])
minus_one_hundred_and_eighty_to_0.sort(key=lambda a:a[2])
zero_to_90.sort(key=lambda a:a[2])
visible_asteroids=ninety_to_180+minus_one_hundred_and_eighty_to_0+zero_to_90
#The 200th destroyed asteroid is:
print(visible_asteroids[199])
#And our solution:
visible_asteroids[199][1][0][0]+visible_asteroids[199][1][0][1]*100

(((22, 17), 258), ((16, 6), 185), 28.61045966596522, 12.529964086141668)


616

In [86]:
visible_asteroids

[(((22, 17), 258), ((19, 17), 225), 90.0, 3.0),
 (((22, 17), 258), ((4, 18), 55), 93.17983011986423, 18.027756377319946),
 (((22, 17), 258), ((5, 18), 62), 93.3664606634298, 17.029386365926403),
 (((22, 17), 258), ((8, 18), 95), 94.08561677997488, 14.035668847618199),
 (((22, 17), 258), ((9, 18), 110), 94.39870535499554, 13.038404810405298),
 (((22, 17), 258), ((1, 19), 22), 95.44033203100551, 21.095023109728988),
 (((22, 17), 258), ((2, 19), 35), 95.71059313749964, 20.09975124224178),
 (((22, 17), 258), ((5, 19), 63), 96.70983680775694, 17.11724276862369),
 (((22, 17), 258), ((14, 18), 168), 97.1250163489018, 8.06225774829855),
 (((22, 17), 258), ((0, 20), 8), 97.76516601842535, 22.20360331117452),
 (((22, 17), 258), ((8, 19), 96), 98.13010235415598, 14.142135623730951),
 (((22, 17), 258), ((9, 19), 111), 98.74616226255522, 13.152946437965905),
 (((22, 17), 258), ((3, 20), 46), 98.9726266148964, 19.235384061671343),
 (((22, 17), 258), ((16, 18), 191), 99.46232220802563, 6.082762530298